# 감정 사전 분석 

## 감정 사전 불러오기 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 감정 사전 로딩 - NRC Emotion Lexicon
nrc_path = "../data/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"
nrc = pd.read_csv(nrc_path, sep="\t", names=["word", "emotion", "association"])

# 감정 연관성 있는 단어만 필터링
nrc = nrc[nrc["association"] == 1]
print(nrc.head())
print("고유 감정 목록:", nrc["emotion"].unique())

In [ ]:
# 감정별 단어 수 집계 시각화
emotion_counts = nrc["emotion"].value_counts().sort_values(ascending=False)

plt.figure(figsize=(8, 5))
sns.barplot(x=emotion_counts.index, y=emotion_counts.values)
plt.title("Emotion Word Counts in NRC Lexicon")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

* NRC 감정 사전은 joy, trust, anticipation 등 비슷한 정서가 분리돼 있어 해석이 애매한 경우가 많음
* 그래서 의미가 가까운 감정들을 love, anger, sadness 같은 감정군으로 묶어 더 직관적으로 분석할 수 있도록 구성함  
* 이 구조는 감정 점수 계산, 트렌드 분석, 클러스터링 등에 그대로 활용할 예정

## 사용자 정의 감정군으로 매핑 

In [ ]:
# NRC 감정 ➜ 사용자 정의 감정군으로 매핑
custom_emotion_map = {
    "joy": "love",
    "trust": "love",
    "positive": "love",

    "sadness": "sadness",
    "fear": "sadness",
    "negative": "sadness",

    "anger": "anger",
    "disgust": "anger",

    "anticipation": "hope",
    "surprise": "surprise",
}

# 감정 단어 데이터프레임에 사용자 정의 감정군 추가
nrc["custom_emotion"] = nrc["emotion"].map(custom_emotion_map)

In [ ]:
# 감정군별 단어 수 집계 시각화 
group_counts = nrc["custom_emotion"].value_counts().sort_values(ascending=False)

plt.figure(figsize=(8, 5))
sns.barplot(x=group_counts.index, y=group_counts.values, palette="muted")
plt.title("Custom Emotion Group Distribution in NRC Lexicon")
plt.ylabel("Number of Words")
plt.xlabel("Emotion Group")
plt.tight_layout()
plt.show()

In [ ]:
# 단어 기준 감정군 set 생성
word_to_emotions = (
    nrc.groupby("word")["custom_emotion"]
    .apply(set)
    .to_dict()
)

# 예시
print(word_to_emotions.get("hug"))  # {'love'}
print(word_to_emotions.get("kill"))  # {'sadness'}
print(word_to_emotions.get("fight"))  # {'anger', 'sadness'}

In [ ]:
# # 딕셔너리 키를 안전하게 문자열로 강제 변환 (strip도 함께 적용)
# word_to_emotions_cleaned = {
#     str(k).strip(): v for k, v in word_to_emotions.items()
# }

## 감정 단어 기반 특징 추출

In [ ]:
# 가사에서 감정군에 속하는 단어 추출하는 함수
def extract_emotion_words(tokens, word_to_emotions):
    # 감정군에 속하는 단어만 추출된 리스트를 return
    return [t for t in tokens if str(t).strip() in word_to_emotions]

In [ ]:
# 감정군에 속하는 단어 비율 구하는 함수
from collections import Counter

def get_emotion_word_counts(tokens, word_to_emotions):
    # 감정 단어들 필터링
    matched = [w for w in tokens if w in word_to_emotions]
    total = len(tokens)

    # 감정들을 풀어서 리스트로 수집 (set → 각 감정)
    emotion_list = []
    for w in matched:
        emotion_list.extend(word_to_emotions[w])  # set → list로 확장

    count = Counter(emotion_list)
    
    # 비율 계산
    proportion = {emotion: round(count[emotion] / total * 100, 2) for emotion in count}
    return count, proportion

In [ ]:
import pandas as pd

# 데이터 불러오기 
df = pd.read_pkl('../data/top100_cleaned.pkl')

In [ ]:
# import ast

# if isinstance(df["lyrics_tokens"].iloc[0], str):
#     df["lyrics_tokens"] = df["lyrics_tokens"].apply(ast.literal_eval)

In [ ]:
# 샘플 가사 2곡 뽑기
sample_df = df.sample(2, random_state=42)

for idx, row in sample_df.iterrows():
    title = row["title"]
    artist = row["artist"]
    tokens = row["lyrics_tokens"]
    total = len(tokens)

    # 감정 단어 추출 및 감정별 빈도/비율 계산
    emotion_words = extract_emotion_words(tokens, word_to_emotions)
    emo_count = len(emotion_words)
    ratio = emo_count / total if total > 0 else 0
    emo_counts, emo_props = get_emotion_word_counts(tokens, word_to_emotions)

    # 출력
    print(f"{title} - {artist}")
    print(f"가사 길이: {total} 토큰")
    print(f"감정 단어 수: {emo_count}")
    print(f"감정 단어 비율: {ratio:.2%}")
    print(f"감정 단어: {emotion_words}")
    print("감정별 단어 수:", dict(emo_counts))
    print("감정별 비율 (%):", emo_props)
    print("-" * 50)

### 샘플로 저장 

In [ ]:
import pandas as pd
from collections import Counter

# 감정 전체 목록 미리 추출
all_emotions = sorted(set(em for emo_set in word_to_emotions.values() for em in emo_set))

sample_data = []
sample_n = 100

for i in range(sample_n):
    row = df_lemmatized.iloc[i]
    tokens = row["lyrics_tokens"]
    title = row.get("title", f"Unknown {i+1}")
    artist = row.get("artist", "Unknown")

    total = len(tokens)
    
    # 감정 단어 추출 및 감정 빈도/비율 계산
    matched_words = extract_emotion_words(tokens, word_to_emotions)
    emotion_counts, emotion_ratios = get_emotion_word_counts(tokens, word_to_emotions)

    entry = {
        "title": title,
        "artist": artist,
        "total_tokens": total,
        "emotion_word_count": len(matched_words),
        "emotion_word_ratio": round(len(matched_words) / total * 100, 2) if total > 0 else 0.0,
        "emotion_words": ", ".join(matched_words),
    }

    # 감정별 개수와 비율 추가
    for emo in all_emotions:
        entry[f"count_{emo}"] = emotion_counts.get(emo, 0)
        entry[f"ratio_{emo}"] = emotion_ratios.get(emo, 0.0)

    sample_data.append(entry)

# 저장
df_sample_stats = pd.DataFrame(sample_data)
df_sample_stats.to_csv("../results/sample_emotion_analysis.csv", index=False)

### 워드클라우드 만들기

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter

# 모든 감정 단어 리스트 수집
all_emotion_words = df_lemmatized["emotion_words"].explode().dropna()

# 단어 빈도 계산
emotion_word_freq = Counter(all_emotion_words)

# 워드클라우드 생성
wc = WordCloud(width=800, height=400, background_color="white").generate_from_frequencies(emotion_word_freq)

# 시각화
plt.figure(figsize=(10, 5))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.title("Emotion Word Cloud")
plt.tight_layout()
plt.show()

In [ ]:
from collections import defaultdict

# 감정군별 워드클라우드 
emo_groups = defaultdict(list)
for tokens in df_lemmatized["lyrics_tokens"]:
    for token in tokens:
        emotions = word_to_emotions.get(token, set())
        for emo in emotions:
            emo_groups[emo].append(token)

## 감정 점수 계산 

In [ ]:
from collections import Counter

def compute_emotion_scores(tokens, word_to_emotions):
    """
    주어진 토큰 리스트에서 감정군별 등장 횟수를 계산하는 함수

    Parameters:
        tokens (list of str): 형태소 분석된 가사 토큰
        word_to_emotions (dict): 단어 → 감정군 set 매핑 딕셔너리

    Returns:
        dict: 감정군별 단어 수 (ex: {'love': 3, 'sadness': 1, ...})
    """
    counter = Counter()

    for token in tokens:
        emotions = word_to_emotions.get(token, set())
        for emo in emotions:
            counter[emo] += 1

    return dict(counter)


In [ ]:
# 샘플 토큰
tokens = ["love", "cry", "hug", "kill", "smile", "trust"]

# 감정 점수 계산
emotion_scores = compute_emotion_scores(tokens, word_to_emotions)

print(emotion_scores)
# 예시 출력: {'love': 4, 'sadness': 2, 'surprise': 1}